In [66]:
import sys, os
import time
import numpy as np
import pandas as pd
import csv

In [67]:
import pyincore

In [68]:
from pyincore import InsecureIncoreClient
from pyincore import InventoryDataset
from pyincore.analyses.populationdislocation.populationdislocation import PopulationDislocation
from pyincore.analyses.populationdislocation.structuredamage import StructureDamage
from pyincore.analyses.buildingdamage.buildingdamage import BuildingDamage
from pyincore.analyses.stochastic_population.stochastic_population_allocation import StochasticPopulationAllocation

In [69]:
cred = ""
client = InsecureIncoreClient("http://incore2-services.ncsa.illinois.edu:8888", cred)

In [70]:
# Path to inventories
blding_inv_path = "input_files/IN-CORE_01av3_SetupSeaside_FourInventories_2018-08-29_buildinginventory.csv"
#addres_inv_path = "input_files/IN-CORE_01av3_SetupSeaside_FourInventories_2018-08-29_addresspointinventory.csv"
#infras_inv_path = "input_files/IN-CORE_01av3_SetupSeaside_FourInventories_2018-08-29_waterinventory.csv"
#popula_inv_path = "input_files/IN-CORE_01av3_SetupSeaside_FourInventories_2018-08-29_popinventory.csv"
block_data_path = "input_files/IN-CORE_01av3_SetupSeaside_FourInventories_2018-08-29_bgdata.csv"
# hazard related
bldshp_inv_path = "input_files/Seaside_buildings/"
hazard_id_service = "earthquake/5ba92505ec23090435209071"
mapping_id_service = "5b47b350337d4a3629076f2c"
damage_ratio_path = "input_files/Building Damage Ratios v1.1.csv"
# temporary
merged_inv_path = "input_files/merged_final_inv_1238.csv"

In [71]:
# Stochastic parameters
initial_seed = 1238
n_iterations = 1
# save intermediate csv files
intermediate_files: bool = False

In [72]:
# Analysis parameter
output_file_path = ""

output_dc_filename = "dislocation_" + str(n_iterations) + ".csv"
output_mc_filename = "mc-results_" + str(n_iterations) + ".csv"

In [73]:
start_time = time.time()

In [74]:
# building shape file (one set only)
shp_file = None
for file in os.listdir(bldshp_inv_path):
    if file.endswith(".shp"):
        shp_file = os.path.join(bldshp_inv_path, file)

building_shp = os.path.abspath(shp_file)
building_set = InventoryDataset(building_shp)

In [75]:
# Building damage
hzdmg = BuildingDamage(client, hazard_id_service, damage_ratio_path, intermediate_files)
building_hazard = hzdmg.get_damage(building_set.inventory_set, mapping_id_service)

# merge hazard building damage probabilities with building inventory
merge_dmg = StructureDamage(output_file_path)
building_inv_loss = merge_dmg.merge_building_damage(blding_inv_path, building_hazard)
# end of Building Damage

In [76]:
# Start of the Stochastic Population Allocation and Dislocation
# final table with dislocation values
mc_out = np.zeros((n_iterations, 11))

In [77]:
seed_i = initial_seed + 0
    
# Stochastic Population Allocation
#stal = StochasticPopulationAllocation(addres_inv_path, blding_inv_path,
#                                      infras_inv_path, popula_inv_path,
#                                      "", "",
#                                      None, None,
#                                      intermediate_files)
#stal.set_building_inv(building_inv_loss)
#merged_inv = stal.get_iteration_stochastic_allocation(seed_i)
    
merged_inv = pd.DataFrame()
if os.path.isfile(merged_inv_path):
    merged_inv = pd.read_csv(merged_inv_path, header="infer")
# end of Stochastic Population Allocation
    
# Population Dislocation
podi = PopulationDislocation(client, output_file_path, intermediate_files)
# combine merged_inventories with block group data
merged_block_inv = podi.merge_block_data(merged_inv, block_data_path)
merged_final_inv = podi.get_dislocation(seed_i, merged_block_inv)

if intermediate_files:
    merged_final_inv.to_csv(output_file_path + "final_inventory_" + str(seed_i) + ".csv", sep=",")

mc_out_iter = podi.aggregate_disl_value(seed_i, merged_final_inv)
# end of Population Dislocation

mc_out[0] = mc_out_iter[:]

In [78]:
# Analyze mc_out and write results to DataFrames and csv tables
output_mc_filename = "mc-out_" + str(n_iterations) + ".csv"
output_dc_filename = "mc-long_" + str(n_iterations) + ".csv"

In [79]:
# mc total dislocation output
mc_out_col = {}
# total = 0, owner = 1, renter = 2, renter (no Critic. Infrast.) = 3, renter (no CI) = 4
cols = ["seed",
        "numprec0", "totdisnumprec0", "numprec1", "totdisnumprec1",
        "numprec2", "totdisnumprec2", "numprec3", "totdisnumprec3",
        "numprec4", "totdisnumprec4"]
for idx in range(mc_out.shape[1]):
    mc_out_col[cols[idx]] = np.int_(mc_out[:, idx])

mc_out_pd = pd.DataFrame(mc_out_col)
mc_out_pd.to_csv(output_file_path + output_mc_filename, sep=",")

# mc average (total and percentage) of total and dislocation outputs
min_count = np.int_(np.nanmin(mc_out, axis=0))
max_count = np.int_(np.nanmax(mc_out, axis=0))
mean_count = np.int_(np.nanmean(mc_out, axis=0))
std_count = np.int_(np.nanstd(mc_out, axis=0))

total_count = np.array([min_count, max_count, mean_count, std_count])

mc_out_pct = np.divide(mc_out, np.roll(mc_out, 1, axis=1), out=np.zeros_like(mc_out),
                       where=mc_out != 0)

mean_pct = np.around(np.nanmean(mc_out_pct, axis=0), decimals=2)
std_pct = np.around(np.nanstd(mc_out_pct, axis=0), decimals=2)
min_pct = np.around(np.nanmin(mc_out_pct, axis=0), decimals=2)
max_pct = np.around(np.nanmax(mc_out_pct, axis=0), decimals=2)

total_pct = np.array([min_pct, max_pct, mean_pct, std_pct])

In [80]:
# Write mc_stats to DataFrame and csv table
cols_total = ["Population", "Min", "Max", "Mean", "Std"]

In [81]:
# total population (no dislocation)
total_count_numprec = np.delete(total_count, [0, 2, 4, 6, 8, 10], axis=1)
total_count_numprec = np.transpose(total_count_numprec)

dc_out_total = {}
for idx in range(np.shape(total_count_numprec)[0]):
    if idx == 0:
        dc_out_total[cols_total[0]] = ["Total", "Owner", "Renter", "Owner (no CI)", "Renter (no CI)"]
    else:
        dc_out_total[cols_total[idx]] = np.int_(total_count_numprec[:, idx - 1])

dc_out_pd_total = pd.DataFrame(dc_out_total)
# write to csv file
dc_out_pd_total.to_csv(output_file_path + output_dc_filename, sep=",")

In [82]:
# number of dislocated population
total_count_disnumprec = np.delete(total_count, [0, 1, 3, 5, 7, 9], axis=1)
total_count_disnumprec = np.int_(np.transpose(total_count_disnumprec))

dc_out_disln = {}
for idx in range(np.shape(total_count_disnumprec)[0]):
    if idx == 0:
        dc_out_disln[cols_total[0]] = ["Dislocated Total", "Dislocated Owner", "Dislocated Renter",
                                       "Dislocated Owner (no CI)", "Dislocated Renter (no CI)"]
    else:
        dc_out_disln[cols_total[idx]] = total_count_disnumprec[:, idx - 1]

dc_out_pd_disln = pd.DataFrame(dc_out_disln)
# append to csv file
dc_out_pd_disln.to_csv(output_file_path + output_dc_filename, mode='a', header=False, sep=",")

In [83]:
# percentage of dislocated population
total_pct_disnumprec = np.delete(total_pct, [0, 1, 3, 5, 7, 9], axis=1)
total_pct_disnumprec = np.transpose(total_pct_disnumprec)

dc_out_dislp = {}
for idx in range(np.shape(total_pct_disnumprec)[0]):
    if idx == 0:
        dc_out_dislp[cols_total[0]] = ["Dislocated Total (%)", "Dislocated Owner (%)",
                                       "Dislocated Renter (%)", "Dislocated Owner (no CI) (%)",
                                       "Dislocated Renter (no CI) (%)"]
    else:
        dc_out_dislp[cols_total[idx]] = total_pct_disnumprec[:, idx - 1]

dc_out_pd_dislp = pd.DataFrame(dc_out_dislp)
# append to csv file
dc_out_pd_dislp.to_csv(output_file_path + output_dc_filename, mode='a', header=False, sep=",")

In [84]:
end_time = time.time()
elapsed = end_time - start_time
print("\nExecution time: {:.2f}".format(elapsed) + "s")


Execution time: 24.51s


In [85]:
# mc total dislocation output
pd.DataFrame(mc_out_col)

,seed,numprec0,totdisnumprec0,numprec1,totdisnumprec1,numprec2,totdisnumprec2,numprec3,totdisnumprec3,numprec4,totdisnumprec4
0,1238,6640,3029,2848,1222,3566,1708,113,40,66,18


In [86]:
# total population (no dislocation)
pd.DataFrame(dc_out_pd_total)

,Population,Min,Max,Mean,Std
0,Total,6640,6640,6640,0
1,Owner,2848,2848,2848,0
2,Renter,3566,3566,3566,0
3,Owner (no CI),113,113,113,0
4,Renter (no CI),66,66,66,0


In [87]:
# number of dislocated population
pd.DataFrame(dc_out_disln)

,Population,Min,Max,Mean,Std
0,Dislocated Total,3029,3029,3029,0
1,Dislocated Owner,1222,1222,1222,0
2,Dislocated Renter,1708,1708,1708,0
3,Dislocated Owner (no CI),40,40,40,0
4,Dislocated Renter (no CI),18,18,18,0


In [88]:
# percentage of dislocated population
pd.DataFrame(dc_out_dislp)

,Population,Min,Max,Mean,Std
0,Dislocated Total (%),0.46,0.46,0.46,0.0
1,Dislocated Owner (%),0.43,0.43,0.43,0.0
2,Dislocated Renter (%),0.48,0.48,0.48,0.0
3,Dislocated Owner (no CI) (%),0.35,0.35,0.35,0.0
4,Dislocated Renter (no CI) (%),0.27,0.27,0.27,0.0
